<a href="https://colab.research.google.com/github/Leena-GO/Leena-GO/blob/main/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A0/%ED%98%BC%EC%9E%90_%EA%B3%B5%EB%B6%80%ED%95%98%EB%8A%94_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%2B_%EB%94%A5%EB%9F%AC%EB%8B%9D_04_%EB%8B%A4%EC%96%91%ED%95%9C_%EB%B6%84%EB%A5%98_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04-1 로지스틱 회귀

## 1. 럭키백의 확률

In [1]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [2]:
#생선 종류 유니크값 확인
fish['Species'].unique()

array(['Bream', 'Roach', 'Whitefish', 'Parkki', 'Perch', 'Pike', 'Smelt'],
      dtype=object)

In [3]:
fish.columns

Index(['Species', 'Weight', 'Length', 'Diagonal', 'Height', 'Width'], dtype='object')

In [4]:
#Species 열을 타깃 데이터로 만들고, 나머지 열은 입력 데이터로 사용
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
fish_input[:5]

array([[242.    ,  25.4   ,  30.    ,  11.52  ,   4.02  ],
       [290.    ,  26.3   ,  31.2   ,  12.48  ,   4.3056],
       [340.    ,  26.5   ,  31.1   ,  12.3778,   4.6961],
       [363.    ,  29.    ,  33.5   ,  12.73  ,   4.4555],
       [430.    ,  29.    ,  34.    ,  12.444 ,   5.134 ]])

In [5]:
#타깃 데이터 만들기
fish_target = fish['Species'].to_numpy()

In [6]:
#train set과 test set 만들기
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state = 42)

In [7]:
# 훈련 세트와 테스트 세트를 표준화 전처리 해주기, 훈련 세트의 통계 값으로 테스트 세트 변환 필요

from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

- 필요 데이터가 모두 정리되었으니,  k-최근접 이웃 분류기로 테스트 세트에 들어있는 확률을 예측해본다.

In [10]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors = 3) #기본값은 5
kn.fit(train_scaled, train_target) #train data로 한번 학습시키기
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


- 앞서 보았던 fish['Species']에는 7가지 종류의 생선이 있었는데, 훈련 세트와 테스트 세트에도 7개의 생선 종류가 들어가 있다.
- 이렇게 타깃 데이터에 2개 이상의 클래스가 포함된 문제를 **다중 분류**(Multiclass classification)이라고 한다.

- 2장에서 만들었던 이진 분류는, 양성 클래스와 음성 클래스를 1과 0으로 구성해야 했는데, 사이킷런은 문자열로 된 타깃값을 그대로 사용할 수 있다.
  - 주의: 타깃값을 그대로 사이킷런 모델에 전달하면, 순서가 자동으로 알파벳 순이 된다.
  - KNeighborsClassifier에 정렬된 타깃값은 **classes_** 속성에 있다.

In [11]:
kn.classes_

array(['Bream', 'Parkki', 'Perch', 'Pike', 'Roach', 'Smelt', 'Whitefish'],
      dtype=object)

In [12]:
#predict() 메서드는 타깃값으로 예측을 출력한다
kn.predict(test_scaled[:5]) #테스트 세트에 있는 처음 5개 샘플의 타깃값 예측

array(['Perch', 'Smelt', 'Pike', 'Perch', 'Perch'], dtype=object)

- 사이킷런의 분류 모델은 **predict_proba()** 메서드로 클래스별 확률값을 반환한다.
- numpy round()함수는 기본으로 소수점 첫째 자리에서 반올림을 하는데, *decimals* 매개변수로 유지할 소수점 아래 자릿수를 지정할 수 있다.

In [13]:
import numpy as np

proba = kn.predict_proba(test_scaled[:5])
np.round(proba, decimals = 4)

array([[0.    , 0.    , 1.    , 0.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 1.    , 0.    ],
       [0.    , 0.    , 0.    , 1.    , 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.6667, 0.    , 0.3333, 0.    , 0.    ],
       [0.    , 0.    , 0.6667, 0.    , 0.3333, 0.    , 0.    ]])

- *predict_proba()* 메서드의 출력 순서는 *classes_* 속성과 같다. 
  - 첫 번째 열이 *Bream*, 두 번째 열이 *Parkki*에 대한 확률



---

- 이 모델이 계산한 확률이 가장 가까운 이웃의 비율이 맞는지 확인해보자.

In [15]:
distance, indexes = kn.kneighbors(test_scaled[3:4])
train_target[indexes]

array([['Roach', 'Perch', 'Perch']], dtype=object)